In [ ]:
"""
!pip install -q sciunit
!pip install -q neo
!pip install -q elephant
!pip install -q gitpython
!pip install -q efel
"""

#!pip install scipy seaborn
!pip install -U git+https://github.com/appukuttan-shailesh/hbp-validation-client.git
#!pip install --quiet fpdf PyPDF2 tabulate
#!pip install ./bbp-services-0.0.15/ ./task-types-0.0.10/ 
!pip install hbp_service_client

In [ ]:
# Create a DUMMY model entry in the ModelCatalog; metadata to be used for registering model
!pip install --upgrade  -q hbp_validation_framework

from hbp_validation_framework import ModelCatalog, TestLibrary
HBP_USERNAME = "pedroernesto"
env="production"
testLibrary = TestLibrary(username=HBP_USERNAME, environment=env)
model_catalog = ModelCatalog.from_existing(testLibrary)

model = model_catalog.register_model(app_id="39968", name="Reconstructed cell's morphology",
                alias="NeuroM-MorphStats-Model", author="Alexander Kozlov", organization="HBP-SP6",
                private=True, cell_type="Fast spiking interneuron", model_type="Single Cell",
                brain_region="Striatum", species="Other",
                instances=[{"source":"https://github.com/pedroernesto/VF_test_models",
                            "version":"1.0", "parameters":""}],
                description="A class to interact with morphology files via the morphometrics-NeuroM's API (morph_stats)")

In [ ]:
# Adding model instances to an existing Model collection in the Model Catalog
import os

from hbp_validation_framework import ModelCatalog, TestLibrary
HBP_USERNAME = "pedroernesto"
env="production"
testLibrary = TestLibrary(username=HBP_USERNAME, environment=env)
model_catalog = ModelCatalog.from_existing(testLibrary)

model_id='b320b49e-fe00-4c91-a0a7-92ed6312e997'
source_dir='http://collab.link/8123/usecases/models/basal_morphology/fs/'

models_path = './str-fs_excerpt'
file_entries = os.listdir(models_path)
morph_files = [f for f in file_entries if f.endswith(".swc")]
for file_name in morph_files:
    morph_name=file_name.split('.')[0]
    instance_id = model_catalog.add_model_instance(model_id=model_id,
                                              source=os.path.join(source_dir,file_name),
                                              version=morph_name,
                                              description="",
                                              parameters='{}',
                                              code_format="")

In [ ]:
# !pip install --upgrade  -q hbp_validation_framework
# !pip uninstall --yes morphounit
# !pip install --upgrade --quiet git+https://github.com/pedroernesto/morphounit.git
# !pip install --quiet /home/pedroernesto/Documents/Project/Code/Models_Validation/Validation/ValidationTests/morphounit

# First login in into the production version of the VF
from hbp_validation_framework import utils, TestLibrary, ModelCatalog
HBP_USERNAME = "pedroernesto"
testLibrary = TestLibrary(username=HBP_USERNAME, environment="production")
modelCatalog = ModelCatalog.from_existing(testLibrary)
# x=ModelCatalog(username=HBP_USERNAME)

In [ ]:
## Grouping (original and scaled) cells with their related clones into same dirs
import os

try:
    from pathlib import Path
except ImportError:
    from pathlib2 import Path  # Python 2 backport
    
models_path = './str-fs_excerpt'
file_entries = os.listdir(models_path)
morph_files = [f for f in file_entries if f.endswith((".swc", 'h5'))]

import shutil      
for file_name in morph_files:
    base_name = os.path.splitext(file_name)[0]
    if 'Clone' in base_name:
        cl_index = base_name.find('_-_Clone')
        dir_name =  base_name[:cl_index]
    else:
        dir_name = base_name
    # Creating a dir for for each orginal or scaled cell
    dir_path = os.path.join(models_path, dir_name)
    Path(dir_path).mkdir(parents=True, exist_ok=True)
    file_path = os.path.join(models_path,file_name)
    shutil.move(file_path, dir_path)

In [ ]:
# !pip uninstall --yes morphounit
# !pip install --quiet /home/pedroernesto/Documents/Project/Code/Models_Validation/Validation/ValidationTests/morphounit

In [1]:
## Testing the morpho-feature values cell by cell
# models_path = './str-fs'
models_path = './str-fs_excerpt_0'
config_path = 'Hippo_cell_clones_morph_stats_config.yaml'

In [ ]:
## Testing the morpho-feature values cell by cell
# Running the test locally, without results registering. No information about the tests will be added to the score 
from morphounit.utils import NeuroM_MorphStats, NeuroM_MorphStats_AddFeatures
import morphounit.tests
import os,json

# pip install path.py

from path import Path
       
# Computing the observation and prediction files
# The non-cloned cells are used as experimental data
obs_file = 'fs_morpho_stats_NO_Field_Diam.json'
pred_file = 'fs_morpho_stats_NO_Field_Diam.json'
dirs_path = sorted(filter(os.path.isdir, Path(models_path).dirs()))

for dir_path in dirs_path:
    file_entries = os.listdir(dir_path)
    morph_files = sorted([f for f in file_entries if f.endswith((".swc", 'h5'))])
    
    for file_name in morph_files:
        morph_path = os.path.join(dir_path, file_name)
        model_name = os.path.splitext(file_name)[0] # Cell-ID


        if morph_files.index(file_name)==0:  # The non-cloned cell (scalled or not)
            print '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%\n', \
                  'Computing observation data for \n', dir_path, '\n', \
                  '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%\n'
            morph_model_obs = NeuroM_MorphStats_AddFeatures(model_name=model_name, morph_path=dir_path, 
                                                            config_path=config_path, morph_stats_file=obs_file)
            observation = morph_model_obs.morph_feature_info
            test = morphounit.tests.NeuroM_MorphStats_Test(observation=observation)
        else:                       # The cloned cells (scalled or not)
            print 'Analysing clone \n', morph_path, '\n'
            morph_model_pred = NeuroM_MorphStats(model_name=model_name, morph_path=morph_path, 
                                                 config_path=config_path, morph_stats_file=pred_file)
            
            # test.generate_prediction(model=morph_model_pred)
            score = test.judge(morph_model_pred, deep_error=True)
            print("Score: ", score.score)
            print("----------------------------------------------")

            """
            if "figures" in score.related_data:
                print("Output files: ")
                for item in score.related_data["figures"]:
                    print(item)
            print("----------------------------------------------")
            """

/home/pedroernesto/miniconda2/lib/python2.7/site-packages/pandas/core/nanops.py:39: UserWarning: The installed version of bottleneck 0.8.0 is not supported in pandas and will be not be used
The minimum supported version is 1.0.0

  ver=ver, min_ver=_MIN_BOTTLENECK_VERSION), UserWarning)


%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Computing observation data for 
./str-fs_excerpt_0/dend-oh140807_A0_idE_axon-mpg141017_a1-2_idC_-_Scale_x1.000_y0.850_z1.000 
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

Analysing clone 
./str-fs_excerpt_0/dend-oh140807_A0_idE_axon-mpg141017_a1-2_idC_-_Scale_x1.000_y0.850_z1.000/dend-oh140807_A0_idE_axon-mpg141017_a1-2_idC_-_Scale_x1.000_y0.850_z1.000_-_Clone_0.h5 

('Score: ', 1.2536788363598255)
----------------------------------------------
Analysing clone 
./str-fs_excerpt_0/dend-oh140807_A0_idE_axon-mpg141017_a1-2_idC_-_Scale_x1.000_y0.850_z1.000/dend-oh140807_A0_idE_axon-mpg141017_a1-2_idC_-_Scale_x1.000_y0.850_z1.000_-_Clone_1.h5 

('Score: ', 1.0480026846473016)
----------------------------------------------
Analysing clone 
./str-fs_excerpt_0/dend-oh140807_A0_idE_axon-mpg141017_a1-2_idC_-_Scale_x1.000_y0.850_z1.000/dend-oh140807_A0_idE_axon-mpg141017_a1-2_idC_-_Scale_x1.000_y

('Score: ', 1.3325726297702409)
----------------------------------------------
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Computing observation data for 
./str-fs_excerpt_0/dend-oh140807_A0_idE_axon-mpg141017_a1-2_idC_-_Scale_x1.000_y0.950_z1.000 
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

Analysing clone 
./str-fs_excerpt_0/dend-oh140807_A0_idE_axon-mpg141017_a1-2_idC_-_Scale_x1.000_y0.950_z1.000/dend-oh140807_A0_idE_axon-mpg141017_a1-2_idC_-_Scale_x1.000_y0.950_z1.000_-_Clone_0.h5 

('Score: ', 0.72966228476515516)
----------------------------------------------
Analysing clone 
./str-fs_excerpt_0/dend-oh140807_A0_idE_axon-mpg141017_a1-2_idC_-_Scale_x1.000_y0.950_z1.000/dend-oh140807_A0_idE_axon-mpg141017_a1-2_idC_-_Scale_x1.000_y0.950_z1.000_-_Clone_1.h5 

('Score: ', 0.59806069408830487)
----------------------------------------------
Analysing clone 
./str-fs_excerpt_0/dend-oh140807_A0_idE_axon-mpg141017_a1-2_idC_-_Scale_x

('Score: ', 1.0631008982339685)
----------------------------------------------
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Computing observation data for 
./str-fs_excerpt_0/dend-oh140807_A0_idE_axon-mpg141017_a1-2_idC_-_Scale_x1.000_y1.100_z1.000 
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

Analysing clone 
./str-fs_excerpt_0/dend-oh140807_A0_idE_axon-mpg141017_a1-2_idC_-_Scale_x1.000_y1.100_z1.000/dend-oh140807_A0_idE_axon-mpg141017_a1-2_idC_-_Scale_x1.000_y1.100_z1.000_-_Clone_0.h5 

('Score: ', 0.55334637165171341)
----------------------------------------------
Analysing clone 
./str-fs_excerpt_0/dend-oh140807_A0_idE_axon-mpg141017_a1-2_idC_-_Scale_x1.000_y1.100_z1.000/dend-oh140807_A0_idE_axon-mpg141017_a1-2_idC_-_Scale_x1.000_y1.100_z1.000_-_Clone_1.h5 

('Score: ', 0.79764371554361801)
----------------------------------------------
Analysing clone 
./str-fs_excerpt_0/dend-oh140807_A0_idE_axon-mpg141017_a1-2_idC_-_Scale_x

('Score: ', 0.74484296380652071)
----------------------------------------------
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Computing observation data for 
./str-fs_excerpt_0/dend-oh140807_A0_idF_axon-mpg141017_a1-2_idC 
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

Analysing clone 
./str-fs_excerpt_0/dend-oh140807_A0_idF_axon-mpg141017_a1-2_idC/dend-oh140807_A0_idF_axon-mpg141017_a1-2_idC_-_Clone_0.h5 

('Score: ', 0.54334315126067045)
----------------------------------------------
Analysing clone 
./str-fs_excerpt_0/dend-oh140807_A0_idF_axon-mpg141017_a1-2_idC/dend-oh140807_A0_idF_axon-mpg141017_a1-2_idC_-_Clone_1.h5 

('Score: ', 0.7050766721941315)
----------------------------------------------
Analysing clone 
./str-fs_excerpt_0/dend-oh140807_A0_idF_axon-mpg141017_a1-2_idC/dend-oh140807_A0_idF_axon-mpg141017_a1-2_idC_-_Clone_10.h5 

('Score: ', 0.50922951111881354)
----------------------------------------------
Analysing clo

In [ ]:
## Testing the morpho-feature values cell by cell
# To run the Test with the related-information registred at the HBP-Collab
from morphounit.utils import NeuroM_MorphStats
import morphounit.test

result_uuids = []

file_entries = os.listdir(models_path)
morph_files = [f for f in file_entries if f.endswith((".swc", 'h5'))]
for file_name in morph_files:
    
    morph_path = os.path.join(models_path, file_name)
    model_name = file_name.split(".")[0] # Cell-ID

    morph_model = NeuroM_MorphStats(model_name=model_name, morph_path=morph_path,
                                   config_path=config_path, morph_stats_file=stats_file)

    print morph_path, "---", file_name, '\n\n'
    # score = test.judge(model2Test, deep_error=True)
    
    # On the Collab (observation path is registered with Test definition on the VF!)
    # It is the model registered on the cell before
    model_uuid="b320b49e-fe00-4c91-a0a7-92ed6312e997" # production
    morph_model.model_uuid = model_uuid
    morph_model.version = model_name     # basename of each morphology file

    collab_id="8123"
    test_alias="morph_stats_Test"
    test_version="3.0"
    """
    result_id = utils.run_test(hbp_username=HBP_USERNAME, environment="production", 
                               model=morph_model, test_alias=test_alias, test_version=test_version,
                               storage_collab_id=collab_id, register_result=True, 
                               client_obj=testLibrary)

    result_uuids.append(result_id)
    """

In [ ]:
report_path = None
if len(result_uuids) > 0:
    print("\nDo you wish to generate a report of the tests executed?")
    print("Enter: y/n")
    choice = raw_input().lower()
    valid_choices = {"yes": True, "y": True, "no": False, "n": False}
    if valid_choices[choice]:
        valid_uuids, report_path = utils.generate_report(hbp_username=HBP_USERNAME, environment="dev", 
                                                         result_list=result_uuids, only_combined=True)

In [ ]:
print report_path
if report_path:
    rel_report_path = os.path.relpath(report_path)
    from IPython.display import IFrame    
    display(IFrame(rel_report_path, width=900, height=650))